In [ ]:
from pathlib import Path
import pandas as pd

# Detectar el path base del proyecto (un nivel arriba del notebook)
notebook_path = Path().resolve()
project_root = notebook_path.parent if notebook_path.name == "notebooks" else notebook_path

# Rutas útiles
ruta_raw = project_root / "data" / "raw" / "upc_alumnos.csv"
ruta_limpio = project_root / "data" / "processed" / "upc_alumnos_limpio.csv"

# Verificación
print(f"📁 Notebook path: {notebook_path}")
print(f"📁 Proyecto raíz: {project_root}")
print(f"📄 Archivo a leer: {ruta_raw}")


📁 Notebook path: C:\Users\Usuario\Desktop\upc-student-data-analysis\notebooks
📁 Proyecto raíz: C:\Users\Usuario\Desktop\upc-student-data-analysis
📄 Archivo a leer: C:\Users\Usuario\Desktop\upc-student-data-analysis\data\raw\upc_alumnos.csv


# 1. Cargar datos


In [4]:
df = pd.read_csv('../data/raw/upc_alumnos.csv')

# 2. Normalización de texto

In [5]:
def normalizar_texto(texto):
    if pd.isna(texto):
        return texto
    return texto.strip().title()

columnas_texto = ['nombres', 'apellido_paterno', 'apellido_materno']
for col in columnas_texto:
    df[col] = df[col].astype(str).apply(normalizar_texto)

df['correo'] = df['correo'].astype(str).str.strip().str.lower()
df['correo_universitario'] = df['correo_universitario'].astype(str).str.strip().str.lower()

# 3. Clasificación de correos

In [6]:
df['dominio_correo'] = df['correo'].apply(lambda x: x.split('@')[-1] if '@' in x else 'desconocido')

### Correos válidos


In [7]:
df['correo_valido'] = df['correo'].str.match(r'^[\w\.-]+@[\w\.-]+\.\w+$')

# 4. Validación de número de teléfono

In [8]:
df['numero'] = df['numero'].astype(str).str.strip()
df['telefono_valido'] = df['numero'].str.match(r'^9\d{8}$')

# 5. Comparación de correos personales vs institucionales

In [9]:
df['usa_correo_institucional'] = df.apply(lambda row: row['correo'] == row['correo_universitario'], axis=1)

# 6. Eliminación de duplicados y registros inválidos

### Eliminar duplicados exactos

In [10]:
df = df.drop_duplicates()

### Eliminar registros sin código

In [11]:
df = df[df['codigo'].notnull() & (df['codigo'].str.strip() != '')]

### Marcar registros incompletos (más de 2 campos vacíos)

In [12]:
df['registro_incompleto'] = df.isnull().sum(axis=1) > 2

### Eliminar registros muy incompletos

In [13]:
df = df[df['registro_incompleto'] == False]

# 7. Guardar archivo limpio

In [ ]:
df.to_csv('../data/processed/upc_alumnos_limpio.csv', index=False)
print("✅ Datos limpios guardados en 'data/processed/upc_alumnos_limpio.csv'")

✅ Datos limpios guardados en 'data/processed/upc_alumnos_limpio.csv'
